<a href="https://colab.research.google.com/github/samayakmalhotra/COVID-19-Analysis-in-R/blob/main/CovidData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt #used for drawing graphs
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.decomposition import PCA #to perform PCA to plot the data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split #spliting data into training and testing set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler #For Standardization
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge,LinearRegression,Lasso
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
covid_dataset=pd.read_csv(r"/content/covid_data.csv")
covid_dataset=covid_dataset.drop(['Sample'], axis=1)

In [ ]:
covid_dataset['Sex']=pd.factorize(covid_dataset.Sex)[0]
covid_dataset['Severity']=pd.factorize(covid_dataset.Severity)[0]

In [ ]:
#Splitting the dataset into X & y
X=covid_dataset.drop('Severity' ,axis=1)
y=covid_dataset['Severity']
print(X.shape) #Gives the number of rows and columns
print(y.shape)

(126, 19474)
(126,)


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.40,random_state=100) #random_state produces same output every time we run it.

In [ ]:
#Creating the cross validation scheme
from sklearn.model_selection import KFold
folds= KFold(n_splits=5,shuffle=True,random_state=100)

In [ ]:
 #Specifying the HyperParameters
 Hyper_params=[{"n_features_to_select":list(range(1,5))}]

In [ ]:
#Performing the Grid Search cross validation
#Specify the model
from sklearn.linear_model import LinearRegression
lm=LinearRegression() #Instantiation
lm.fit(X_train,y_train)

LinearRegression()

In [ ]:
from sklearn.svm import SVC

In [ ]:
kernel=['linear','rbf','poly','sigmoid']
for i in kernel:
  model=SVC(kernel=i,C=1.0)
  model.fit(X_train,y_train)
  print('for kernel:',i)
  print('The Accuracy is:',model.score(X_test,y_test))


for kernel: linear
The Accuracy is: 0.7254901960784313
for kernel: rbf
The Accuracy is: 0.6666666666666666
for kernel: poly
The Accuracy is: 0.5882352941176471
for kernel: sigmoid
The Accuracy is: 0.6470588235294118


In [ ]:
model=SVC()
model.fit(X_train,y_train)
print('The Accuracy on testing data is:',model.score(X_test,y_test))
print('The Accuracy on training data is:',model.score(X_train,y_train))


The Accuracy on testing data is: 0.6666666666666666
The Accuracy on training data is: 0.7466666666666667


In [ ]:
#Performing Grid Search
from sklearn.model_selection import GridSearchCV
param_grid={'C':[0.1,100,1000],'kernel':['linear','rbf','poly','sigmoid'],'degree':[1,2,3,4,5,6]}
grid=GridSearchCV(SVC(),param_grid)
grid.fit(X_train,y_train) #Fitting the Model

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 100, 1000], 'degree': [1, 2, 3, 4, 5, 6],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']})

In [ ]:
print(grid.best_params_)
print(grid.score(X_test,y_test)) #Prints accuracy after Grid Search

{'C': 100, 'degree': 3, 'kernel': 'poly'}
0.7254901960784313


In [ ]:
#Outputting the results
cvresults=pd.DataFrame(grid.cv_results_)
cvresults

In [ ]:
print(np.mean(cvresults))

mean_fit_time          0.233168
std_fit_time           0.011382
mean_score_time        0.196450
std_score_time         0.012767
param_C              366.700000
param_degree           3.500000
split0_test_score      0.730556
split1_test_score      0.736111
split2_test_score      0.711111
split3_test_score      0.682407
split4_test_score      0.574074
mean_test_score        0.686852
std_test_score         0.113413
rank_test_score       32.458333
dtype: float64


In [ ]:
#Feature Scaling before PCA is required. If we dont scale it then accuracy would reduce.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_scaled = sc.fit_transform(X)
X_scaled

In [ ]:
#PCA(Principal Component Analysis) to reduce the number of variables and to find the correlation between the variables.
from sklearn.decomposition import PCA

pca = PCA(n_components=10) #n_components = 10 #0.95 tells PCA to retain 95% useful features and create new dimensions.
X_pca = pca.fit_transform(X_scaled)
X_pca.shape

(126, 10)

In [ ]:
pca.explained_variance_ratio_

array([0.28501431, 0.15097841, 0.03709269, 0.02952083, 0.0283656 ,
       0.01984677, 0.01860763, 0.01578315, 0.0142702 , 0.01183295])

In [ ]:
X_train_pca,X_test_pca,y_train,y_test =train_test_split(X_pca,y,test_size=0.33, random_state=30)

In [ ]:
model=LogisticRegression()
model.fit(X_train_pca,y_train)
model.score(X_test_pca,y_test)

0.8095238095238095

In [ ]:
from sklearn.metrics import classification_report

predicted = grid.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)
#report = classification_report(y_test, predicted)#print(report)

In [ ]:
#Finding the Score on test data
import sklearn
r2=sklearn.metrics.r2_score(y_test,y_pred)
print(r2)